<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment2/blob/main/partB/src/Assignment2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting drive to store dataset
from google.colab import drive

drive.mount('/content/gdrive')
#!rm -rf inaturalist_12K

Mounted at /content/gdrive


In [ ]:
%%time
# %cd gdrive/MyDrive/assignments/cs6910/A2/Data
# !pwd
!cp gdrive/MyDrive/assignments/cs6910/A2/Data/inaturalist_12K.zip .
#!gdown --id 11SGStqp8Vug2GDzSpJDwQYHThLIjZFQn
!unzip -q inaturalist_12K.zip
!ls

# import zipfile
# import concurrent.futures

# zf = zipfile.ZipFile('inaturalist_12K.zip')

# def unzip(file):
#     zf.extract(file)

# with concurrent.futures.ProcessPoolExecutor() as executor:
#     executor.map(unzip, zf.infolist())

gdrive	inaturalist_12K  inaturalist_12K.zip  sample_data
CPU times: user 2.88 s, sys: 484 ms, total: 3.36 s
Wall time: 8min 54s


In [ ]:
# !pip install split-folders

In [ ]:
# import splitfolders

# # Splitting the training data into training and validation set
# splitfolders.ratio('./inaturalist_12K/train', output='./inaturalist_12K/output', seed=1337, ratio=(.9, .1), group_prefix=None)

In [ ]:
import os
def print_count_classes_in_valid():
  class_count_valid = {}
  for subdir, dirs, files in os.walk('./inaturalist_12K/val'):
      for file in files:
        class_count_valid[subdir] = class_count_valid.get(subdir,0)+1

  print(class_count_valid)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, \
GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [ ]:
from tensorflow.keras import regularizers

In [ ]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

     |████████████████████████████████| 2.1MB 10.3MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 163kB 39.5MB/s 
     |████████████████████████████████| 133kB 38.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=0546925bdd44978c40fd6996d23cdd7350a0f799ae1d1490229787922f40a655
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=e893aa73e40adebcd18297ec81402b1ac3a4d49254d505325b9e1d81130dd144
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from tensorflow.keras import layers

# Model for resizing and rescaling images
image_rescale = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255)
])

# Model for performing random transformations for data augmentation
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomTranslation(0.2, 0.2),
    layers.experimental.preprocessing.RandomZoom(0.2, 0.2),
    layers.experimental.preprocessing.RandomContrast(0.2)
])

def prepare_data(data_path, inp_img_shape, batch_size, img_preprocess, data_augmentation, data_augment_yes = False, shuffle = True):
    AUTOTUNE = tf.data.AUTOTUNE
    dataset = image_dataset_from_directory(
        data_path, labels='inferred', color_mode='rgb', batch_size=batch_size, image_size=inp_img_shape[:-1], shuffle=shuffle,
        seed=123, label_mode='categorical'
    )
    
    dataset = dataset.map(lambda x, y: (img_preprocess(x), y), num_parallel_calls=AUTOTUNE)

    # Use data augmentation only if data_augment_yes == True (Training set only requires data augmentation)
    if data_augment_yes:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on datasets
    return dataset.prefetch(buffer_size=AUTOTUNE)


def data_generator(inp_img_shape, batch_size, data_augment_yes = False, train_data_path = None, val_data_path = None, test_data_path = None):
    train_data = None
    if train_data_path is not None:
        train_data = prepare_data(train_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, data_augment_yes)
    val_data = None
    if val_data_path is not None:
        val_data = prepare_data(val_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, False)
    test_data = None
    if test_data_path is not None:
        test_data = prepare_data(test_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, False)
    
    return train_data, val_data, test_data


In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
def train_model(model, train_data, loss_function, optimizer = 'adam', learning_rate = 1e-3, epochs = 10, val_data = None):
    if optimizer == 'adam':
        model.compile(optimizer = Adam(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'momentum':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'rmsprop':
        model.compile(optimizer = RMSprop(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'nesterov':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9, nesterov = True), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'nadam':
        model.compile(optimizer = Nadam(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])
    else:
        model.compile(optimizer = SGD(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])

    model.fit(train_data,
              epochs = epochs, 
              validation_data = val_data,
              verbose = 2,
              callbacks = [WandbCallback(monitor='val_accuracy'), EarlyStopping(monitor='val_accuracy', patience=5)])
    
    return model


In [ ]:
from tensorflow.keras.models import Model
get_pretrained_function = {
    'Xception' :          (lambda : tf.keras.applications.xception.Xception(weights='imagenet', include_top=False)),
    'ResNet50' :          (lambda : tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False)),
    'InceptionV3' :       (lambda : tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False)),
    'InceptionResNetV2' : (lambda : tf.keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False))
}

get_k_value = {
    'Xception' :          116,
    'ResNet50' :          143,
    'InceptionV3' :       249,
    'InceptionResNetV2' : 617
}

In [ ]:
def get_pretrained_model(model_name='InceptionResNetV2', no_neurons_dense=1024, k_value=617, no_classes=10):
  pretrained_model = get_pretrained_function[model_name]()
  # add a global spatial average pooling layer
  x = pretrained_model.output
  x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(no_neurons_dense, activation='relu')(x)
  # and a logistic layer -- let's say we have 200 classes
  predictions = Dense(no_classes, activation='softmax')(x)
  model = Model(inputs=pretrained_model.input, outputs=predictions)
  
  for layer in pretrained_model.layers:
    layer.trainable = False

  #for i, layer in enumerate(pretrained_model.layers):
  #  print(i, layer.name)

  for layer in model.layers[:k_value]:
    layer.trainable = False
  for layer in model.layers[k_value:]:
    layer.trainable = True

  return model

In [ ]:
def test_model(model, test_data):
    assert(test_data is not None)
    test_loss, test_accuracy = model.evaluate(test_data, use_multiprocessing = True, workers = 4)
    test_accuracy = round(test_accuracy*100, 2)
    test_loss = round(test_loss, 4)
    print(f'Test Accuracy : {test_accuracy} | Test Loss : {test_loss}')

    return test_loss, test_accuracy


In [ ]:
def CNN_pretrained_train(model, inp_img_shape, train_data_path, config, no_classes = 10, val_data_path = None, test_data_path = None, 
                         wandb_init = True):
    
    id = ''
    if wandb_init:
        id = wandb.util.generate_id()
        run = wandb.init(id = id, project="assignment2", entity="abisheks", reinit=True, config=config)
        
    tf.keras.backend.clear_session()
    
    train_data, val_data, test_data = data_generator(inp_img_shape, config['batch_size'], config['data_augmented'], 
                                                     train_data_path, val_data_path, test_data_path)
    model = train_model(model, train_data, config['loss_function'], config['optimizer'], config['learning_rate'], config['epochs'], val_data)
    
    if test_data is not None:
        test_loss, test_accuracy = test_model(model, test_data)
        wandb.log({'test_accuracy': test_accuracy, 'test_loss': test_loss})

    if wandb_init:
        run.finish()

    return model, id


In [ ]:
# Hyperparameters for building the model for Part-A

inp_img_shape_2 = (224, 224, 3)                                 # Shape of input image from data
no_classes_2 = 10                                               # Number of output classes in the classification problem


config_2 = {
    "learning_rate": 1e-3,                                      # Hyperparameter for updating the parameters in gradient descent
    "epochs": 10,                                               # Number of epochs to train the model   
    "optimizer": 'nesterov',                                    # Gradient descent algorithm used for the parameter updation
    "batch_size": 64,                                           # Batch size used for the optimizer
    "loss_function": 'categorical_crossentropy',                # Loss function used in the optimizer
    "no_neurons_dense": 512,                                    # Number of neurons in the dense FC layer
    "data_augmented": False,                                    # True : Data augmentation is done during training, False : No data augmentation done
    "model_name": 'InceptionResNetV2'                           # Name of the pretrained model
}


# PART-B, Question 1 -- Fine tuning a model pre-trained on Imagenet to iNaturalist dataset
# model = get_pretrained_model(config_2['model_name'], config_2['no_neurons_dense'], get_k_value[config_2['model_name']], no_classes_2)
# modelA, _ = CNN_pretrained(model, inp_img_shape_2, './inaturalist_12K/train', config_2, no_classes_2, './inaturalist_12K/val', './inaturalist_12K/test')

In [ ]:
sweep_config = {
    'name': 'CNN_pretrained',
    'method': 'bayes',                   # Possible search : grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model_name': {
            'values': ['InceptionResNetV2', 'InceptionV3', 'ResNet50' , 'Xception']
        },
        'data_augmented': {
            'values': [True, False]
        },
        'no_neurons_dense': {
            'values': [512,1024]
        },
        'optimizer': {
            'values': ['adam', 'nesterov', 'rmsprop']
        }
    }
}

In [ ]:
def sweep_wrapper(data_path = './inaturalist_12K'):
  
    # Wrapper function to call the CNN pretrained function for sweeping with different hyperparameters

    # Initialize a new wandb run
    run = wandb.init(config=config_2, reinit=True)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    #print(config)

    wandb.run.name = f'mn_{config.model_name}_nd_{config.no_neurons_dense}_op_{config.optimizer}'
    wandb.run.name += '_da' if config.data_augmented else '' 
    wandb.run.save()
    print(wandb.run.name)
    model = get_pretrained_model(config.model_name, config.no_neurons_dense, get_k_value[config.model_name], no_classes_2)
    # Sweep uses L2 regularisation as default as given in the question
    modelA, _ = CNN_pretrained_train(model, inp_img_shape_2, f'{data_path}/train', config_2, no_classes_2, f'{data_path}/val', wandb_init = False)
    run.finish()

In [ ]:
# sweep_id = wandb.sweep(sweep_config, entity="abisheks", project="assignment2")
sweep_id = "abisheks/assignment2/nhoqsu5c"
wandb.agent(sweep_id, lambda : sweep_wrapper())

wandb: Agent Starting Run: m8ziujea with config:
wandb: 	data_augmented: False
wandb: 	model_name: Xception
wandb: 	no_neurons_dense: 512
wandb: 	optimizer: rmsprop
wandb: Currently logged in as: abisheks (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


mn_Xception_nd_512_op_rmsprop
83689472/83683744 [==============================] - 1s 0us/step
Found 9006 files belonging to 10 classes.
Found 1004 files belonging to 10 classes.
Epoch 1/10
141/141 - 169s - loss: 2.0527 - accuracy: 0.4013 - val_loss: 1.4478 - val_accuracy: 0.6096
Epoch 2/10
141/141 - 111s - loss: 1.3287 - accuracy: 0.6841 - val_loss: 0.9466 - val_accuracy: 0.7032
Epoch 3/10
141/141 - 110s - loss: 0.9491 - accuracy: 0.7206 - val_loss: 0.8717 - val_accuracy: 0.7151
Epoch 4/10
141/141 - 110s - loss: 0.8210 - accuracy: 0.7455 - val_loss: 0.8351 - val_accuracy: 0.7291
Epoch 5/10
141/141 - 110s - loss: 0.7518 - accuracy: 0.7582 - val_loss: 0.8069 - val_accuracy: 0.7371
Epoch 6/10
141/141 - 110s - loss: 0.7016 - accuracy: 0.7700 - val_loss: 0.7840 - val_accuracy: 0.7470
Epoch 7/10
141/141 - 110s - loss: 0.6578 - accuracy: 0.7855 - val_loss: 0.7640 - val_accuracy: 0.7580
Epoch 8/10
141/141 - 109s - loss: 0.6210 - accuracy: 0.7942 - val_loss: 0.7508 - val_accuracy: 0.7679
Epoch

epoch,9
loss,0.55771
accuracy,0.81535
val_loss,0.73091
val_accuracy,0.76892
_runtime,1170
_timestamp,1617637449
_step,9
best_val_accuracy,0.77092
best_epoch,8


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▃▂▂▂▁▁▁▁
accuracy,▁▆▆▇▇▇▇███
val_loss,█▃▂▂▂▂▁▁▁▁
val_accuracy,▁▅▆▆▇▇▇███
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


...

epoch,9
loss,0.04278
accuracy,0.99556
val_loss,0.86921
val_accuracy,0.77888
_runtime,864
_timestamp,1617638325
_step,9
best_val_accuracy,0.77888
best_epoch,6


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▃▂▂▁▁▁▁
accuracy,▁▄▅▆▇▇████
val_loss,▇▃▁▁▂▂▄▆▇█
val_accuracy,▁▅▆▆▆█████
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: b7db5tvt with config:
wandb: 	data_augmented: False
wandb: 	model_name: InceptionResNetV2
wandb: 	no_neurons_dense: 512
wandb: 	optimizer: nesterov


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


mn_InceptionResNetV2_nd_512_op_nesterov
219062272/219055592 [==============================] - 7s 0us/step
Found 9006 files belonging to 10 classes.
Found 1004 files belonging to 10 classes.
Epoch 1/10
141/141 - 160s - loss: 1.4275 - accuracy: 0.5898 - val_loss: 0.7939 - val_accuracy: 0.7689
Epoch 2/10
141/141 - 138s - loss: 0.7432 - accuracy: 0.7770 - val_loss: 0.6986 - val_accuracy: 0.7938
Epoch 3/10
